In [1]:
import pandas as pd

seasons = ['2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23']

In [2]:
def access_season_player_data(season: str, previous_season: str, encoding: str) -> pd.DataFrame:
    players_df = pd.read_csv(f'data/{season}/players_raw.csv')[[
        'first_name', 'second_name', 'id', 'team', 'total_points', 'minutes', 'selected_by_percent', 
        'now_cost', 'bps', 'goals_scored', 'assists'
    ]]
    gw1_previous_df = pd.read_csv(f'data/{season}/gws/gw1.csv', encoding=encoding)[[
        'element', 'value'
    ]]
    gw1_previous_df.rename(columns={'value': 'previous_start_value'}, inplace=True)
    gw1_next_df = pd.read_csv(f'data/{previous_season}/gws/gw1.csv', encoding=encoding)[[
        'name', 'value'
    ]]
    gw1_next_df.rename(columns={'value': 'target_value'}, inplace=True)
    gw1_next_df['name'] = gw1_next_df['name'].str.lower().str.replace('_', '').str.replace(' ', '') \
        .str.replace('0', '') \
        .str.replace('1', '') \
        .str.replace('2', '') \
        .str.replace('3', '') \
        .str.replace('4', '') \
        .str.replace('5', '') \
        .str.replace('6', '') \
        .str.replace('7', '') \
        .str.replace('8', '') \
        .str.replace('9', '') 
    teams_df = pd.read_csv(f'webscraper_output/teams_webscraper.csv')
    teams_df = teams_df[teams_df['season'] == season]
    position_df = pd.read_csv('data/cleaned_merged_seasons.csv')[['position', 'element', 'season_x']]
    position_df['unique_id'] = position_df['element'].astype(str)  + '-' + position_df['season_x']
    position_df = position_df[position_df['season_x'] == season]
    combined = players_df.merge(gw1_previous_df, how='left', left_on='id', right_on='element').merge(teams_df, how='left', left_on='team', right_on='team_id') 
    combined['unique_id'] = combined['id'].astype(str)  + '-' + combined['season']
    combined['full_name'] = combined['first_name'].str.lower() + combined['second_name'].str.lower()
    final = combined.merge(position_df, how='left', on='unique_id').merge(gw1_next_df, how='left', left_on='full_name', right_on='name')
    final.rename(columns={'now_cost': 'previous_end_value'}, inplace=True)
    final['delta_value'] = final['previous_end_value'] - final['previous_start_value']
    final = final[[
        'unique_id', 'full_name', 'position', 'teams', 'minutes', 'goals_scored', 'assists', 
        'bps', 'selected_by_percent', 'previous_start_value', 'delta_value', 'target_value'
    ]]
    final = final.dropna().drop_duplicates()
    final[['minutes', 'goals_scored', 'assists', 'bps']] = final[['minutes', 'goals_scored', 'assists', 'bps']].astype(float)
    
    return final

In [3]:
access_season_player_data('2017-18', '2018-19', 'latin-1').columns

/opt/anaconda3/envs/fpl_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (4,20) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Index(['unique_id', 'full_name', 'position', 'teams', 'minutes',
       'goals_scored', 'assists', 'bps', 'selected_by_percent',
       'previous_start_value', 'delta_value', 'target_value'],
      dtype='object')

In [4]:
player_df = pd.DataFrame(columns=['unique_id', 'full_name', 'position', 'teams', 'minutes',
       'goals_scored', 'assists', 'bps', 'selected_by_percent',
       'previous_start_value', 'delta_value', 'target_value'])
for i in range(len(seasons)-1):
    player_df = player_df.append(access_season_player_data(seasons[i+1], seasons[i], 'latin-1'), ignore_index=True)
    
player_df['position'] = player_df['position'].replace('GKP', 'GK')
player_df_encoded = pd.get_dummies(player_df, columns=['position', 'teams'])
player_df_encoded


/opt/anaconda3/envs/fpl_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (4,20) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/opt/anaconda3/envs/fpl_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (4,20) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/opt/anaconda3/envs/fpl_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (4,20) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/opt/anaconda3/envs/fpl_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (4,20) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_

,unique_id,full_name,minutes,goals_scored,assists,bps,selected_by_percent,previous_start_value,delta_value,target_value,...,teams_nottinghamforest,teams_sheffieldunited,teams_southampton,teams_stokecity,teams_swanseacity,teams_tottenhamhotspur,teams_watford,teams_westbromwichalbion,teams_westhamunited,teams_wolverhamptonwanderers
0,7-2017-18,héctorbellerín,3050.0,2.0,4.0,656.0,10.3,60.0,-1.0,65.0,...,0,0,0,0,0,0,0,0,0,0
1,11-2017-18,robholding,819.0,0.0,1.0,196.0,0.7,50.0,-6.0,45.0,...,0,0,0,0,0,0,0,0,0,0
2,16-2017-18,mesutözil,2161.0,4.0,9.0,506.0,2.1,95.0,-2.0,95.0,...,0,0,0,0,0,0,0,0,0,0
3,18-2017-18,aaronramsey,1844.0,7.0,10.0,503.0,4.5,70.0,-2.0,80.0,...,0,0,0,0,0,0,0,0,0,0
4,21-2017-18,alexiwobi,1827.0,3.0,5.0,426.0,1.0,55.0,-4.0,60.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408,484-2022-23,maxkilman,3307.0,0.0,1.0,622.0,3.1,45.0,-2.0,45.0,...,0,0,0,0,0,0,0,0,0,1
1409,485-2022-23,lukecundle,0.0,0.0,0.0,0.0,0.2,45.0,-1.0,45.0,...,0,0,0,0,0,0,0,0,0,1
1410,488-2022-23,fabiosilva,0.0,0.0,0.0,0.0,0.0,55.0,0.0,60.0,...,0,0,0,0,0,0,0,0,0,1
1411,516-2022-23,nathancollins,1811.0,0.0,0.0,326.0,0.5,45.0,-3.0,45.0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
player_df_encoded.describe()

,minutes,goals_scored,assists,bps,selected_by_percent,previous_start_value,delta_value,target_value,position_DEF,position_FWD,...,teams_nottinghamforest,teams_sheffieldunited,teams_southampton,teams_stokecity,teams_swanseacity,teams_tottenhamhotspur,teams_watford,teams_westbromwichalbion,teams_westhamunited,teams_wolverhamptonwanderers
count,1413.000000,1413.000000,1413.000000,1413.000000,1413.000000,1413.000000,1413.000000,1413.000000,1413.000000,1413.000000,...,1413.000000,1413.000000,1413.000000,1413.000000,1413.000000,1413.000000,1413.000000,1413.000000,1413.000000,1413.000000
mean,1500.454352,2.174805,1.904459,281.950460,3.141118,54.585987,-1.682944,55.109696,0.361642,0.131635,...,0.005662,0.014862,0.070771,0.002831,0.002123,0.067233,0.021939,0.011323,0.065110,0.022647
std,1072.056717,4.013065,2.825935,219.672668,6.193978,13.386624,2.504947,13.405793,0.480646,0.338213,...,0.075058,0.121043,0.256533,0.053149,0.046045,0.250513,0.146537,0.105845,0.246807,0.148828
min,0.000000,0.000000,0.000000,0.000000,0.000000,40.000000,-10.000000,40.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,517.000000,0.000000,0.000000,89.000000,0.200000,45.000000,-3.000000,45.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1532.000000,1.000000,1.000000,261.000000,0.700000,50.000000,-2.000000,50.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2436.000000,2.000000,3.000000,443.000000,3.000000,60.000000,-1.000000,60.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3420.000000,30.000000,17.000000,1008.000000,44.500000,130.000000,16.000000,130.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
player_df_encoded.corr()

,minutes,goals_scored,assists,bps,selected_by_percent,previous_start_value,delta_value,target_value,position_DEF,position_FWD,...,teams_nottinghamforest,teams_sheffieldunited,teams_southampton,teams_stokecity,teams_swanseacity,teams_tottenhamhotspur,teams_watford,teams_westbromwichalbion,teams_westhamunited,teams_wolverhamptonwanderers
minutes,1.000000,0.400772,0.472474,0.943888,0.482788,0.274707,0.238470,0.192924,0.059437,-0.047420,...,-0.018550,0.006006,0.012779,0.071981,0.049191,0.018570,0.036196,0.018978,0.022369,0.002199
goals_scored,0.400772,1.000000,0.612102,0.467349,0.666239,0.751797,0.328282,0.685304,-0.273659,0.387946,...,-0.019747,-0.038885,-0.021656,0.000999,-0.009675,0.062974,0.003108,-0.009665,-0.021510,-0.023234
assists,0.472474,0.612102,1.000000,0.543811,0.599994,0.601400,0.275610,0.527925,-0.169551,0.134690,...,-0.000787,-0.047607,-0.035605,0.011233,0.001560,0.060100,-0.029139,-0.027161,0.000802,-0.031898
bps,0.943888,0.467349,0.543811,1.000000,0.624482,0.344863,0.366510,0.266417,0.095739,-0.054818,...,-0.026184,-0.030735,-0.019719,0.068132,0.047272,0.061949,0.005710,-0.011946,0.009021,0.000468
selected_by_percent,0.482788,0.666239,0.599994,0.624482,1.000000,0.519094,0.624871,0.447971,-0.034829,0.119559,...,-0.005223,-0.045213,-0.054961,0.002013,0.012358,0.088634,-0.037043,-0.029986,-0.046042,0.012741
previous_start_value,0.274707,0.751797,0.601400,0.344863,0.519094,1.000000,0.090615,0.913550,-0.327837,0.395285,...,-0.029384,-0.028980,-0.064673,-0.003329,-0.004318,0.125514,-0.038690,-0.019181,-0.044353,-0.059276
delta_value,0.238470,0.328282,0.275610,0.366510,0.624871,0.090615,1.000000,0.082480,0.064695,-0.015860,...,-0.017088,-0.097303,-0.032739,0.019851,0.031001,0.067580,0.000330,-0.050946,-0.043724,0.051015
target_value,0.192924,0.685304,0.527925,0.266417,0.447971,0.913550,0.082480,1.000000,-0.329304,0.403715,...,-0.011175,-0.059926,-0.067128,-0.005406,0.005359,0.131713,-0.037278,-0.030823,-0.036408,-0.079339
position_DEF,0.059437,-0.273659,-0.169551,0.095739,-0.034829,-0.327837,0.064695,-0.329304,1.000000,-0.293050,...,-0.017533,0.004936,-0.029662,0.015343,-0.002718,0.044960,-0.022231,-0.024867,0.016292,0.024033
position_FWD,-0.047420,0.387946,0.134690,-0.054818,0.119559,0.395285,-0.015860,0.403715,-0.293050,1.000000,...,-0.001481,0.055976,0.031316,-0.020745,-0.017959,-0.046018,0.041717,0.037467,-0.017905,-0.017057


In [5]:
player_df.to_csv('player_df.csv')
player_df_encoded.to_csv('player_df_dummies.csv')